In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def visualize(img, faces, similar_index, thickness = 2):
    if faces[1] is not None:
        for idx, face in enumerate(faces[1]):
            coords = face[:-1].astype(np.int32)
            if idx == similar_index:
                cv2.rectangle(img, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 255, 0), thickness)
            else:
                cv2.rectangle(img, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 0, 255), thickness)
       

In [3]:
def matcher(detector, recognizer,aligned_face1 ,img2, l2_threshold = 1.128):
    index = -1
    aligned_faces2 = []
    
    detector.setInputSize((img2.shape[1], img2.shape[0]))

    faces = detector.detect(img2)  
    
    if faces[1] is not None:
        for face in faces[1]:
            alignCrop = recognizer.alignCrop(img2, face)
            aligned_faces2.append(alignCrop)
            
    feature1 = recognizer.feature(aligned_face1)
    for i, face2 in enumerate (aligned_faces2):
        feature2 = recognizer.feature(face2)
        score = recognizer.match(feature1, feature2, cv2.FACE_RECOGNIZER_SF_FR_NORM_L2)
        if score<=l2_threshold:
            index = i
            
    visualize(img2, faces, index)  

In [6]:
detector = cv2.FaceDetectorYN_create("Model/face_detection_yunet_2022mar.onnx","",
    (320, 320),
    0.8,
    0.3,
    5000)

recognizer = cv2.FaceRecognizerSF_create("Model/face_recognition_sface_2021dec.onnx", "")

img1 = cv2.imread('Messi.jpg')


detector.setInputSize((img1.shape[1], img1.shape[0]))

faces1 = detector.detect(img1)

aligned_face1 = recognizer.alignCrop(img1, faces1[1])

cap = cv2.VideoCapture('MessiChampion.mp4')

while True:
    flag, frame = cap.read()
    frame = cv2.flip(frame, 1)    
    matcher(detector, recognizer, aligned_face1, frame)
        
    cv2.imshow('FaceRecognition', frame)
    
    if cv2.waitKey(1) & 0xff == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
plt.imshow(img1[..., ::-1])